Import des librairies nécessaires: 

In [1]:
import pandas as pd
import openpyxl
import os

Intégration et nettoyage des données: 

In [2]:
## Définition de fonction d'intégration
def charger(path, sep=";"):
    df = pd.read_csv(path, sep=sep)
    return df

def nettoyer(df):
    df = df.drop_duplicates()
    df = df.fillna("NA")
    return df

def sauvegarder(df, path):
    os.makedirs(os.path.dirname(path), exist_ok=True)
    df.to_csv(path, index=False)


In [4]:
## Chargement des données
data = nettoyer(charger("../DS_DEVDUR_data.csv")) 
meta = nettoyer(charger("../DS_DEVDUR_metadata.csv"))

Transformation des données: 

In [5]:
## Création des tables de correspondances

def tables_correspondances(meta):
    tables = {}
    variables = meta['COD_VAR'].unique()

    for var in variables:
        sous_table = meta[meta['COD_VAR'] == var]
        correspondance = sous_table[['COD_MOD', 'LIB_MOD']].drop_duplicates()
        tables[var] = correspondance
    return tables

def sauvegarder_tables(tables, dossier_sortie):
    os.makedirs(dossier_sortie, exist_ok=True)

    for var, table in tables.items():
        nom_fichier = f"{var}_correspondance.csv"
        chemin = os.path.join(dossier_sortie, nom_fichier)
        table.to_csv(chemin, index=False)
        print(f"Correspondances '{var}' enregistrée : {chemin}")

In [7]:
correspondances = sauvegarder_tables(tables_correspondances(meta), "../DATA")

Correspondances 'AGE' enregistrée : ../DATA/AGE_correspondance.csv
Correspondances 'DEVDUR_COMPOSITE' enregistrée : ../DATA/DEVDUR_COMPOSITE_correspondance.csv
Correspondances 'EMPSTA' enregistrée : ../DATA/EMPSTA_correspondance.csv
Correspondances 'INDICATEUR_DEVDUR' enregistrée : ../DATA/INDICATEUR_DEVDUR_correspondance.csv
Correspondances 'OBS_STATUS' enregistrée : ../DATA/OBS_STATUS_correspondance.csv
Correspondances 'ODD' enregistrée : ../DATA/ODD_correspondance.csv
Correspondances 'PCS' enregistrée : ../DATA/PCS_correspondance.csv
Correspondances 'SEX' enregistrée : ../DATA/SEX_correspondance.csv
Correspondances 'TIME_PERIOD' enregistrée : ../DATA/TIME_PERIOD_correspondance.csv
Correspondances 'TYPE_INDICATEUR_DEVDUR' enregistrée : ../DATA/TYPE_INDICATEUR_DEVDUR_correspondance.csv
Correspondances 'UNIT_MEASURE' enregistrée : ../DATA/UNIT_MEASURE_correspondance.csv
Correspondances 'UNIT_MULT' enregistrée : ../DATA/UNIT_MULT_correspondance.csv
Correspondances 'GEO' enregistrée : ..

Intégration des données à l'excel résultat

In [ ]:
from openpyxl import Workbook

path_file = '../OUTPUT/TDB_ODD.xlsx'

# Si le fichier n'existe pas, on le crée
if not os.path.exists(path_file):
    wb = Workbook()
    wb.save(path_file)

# Ensuite, on écrit dans le fichier
with pd.ExcelWriter(path_file, mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
    data.to_excel(writer, sheet_name='DATA', index=False)
    meta.